# Covid-19 Data Aggregation

This notebook is in conjunction with the ongoing research on Covid-19 and human mobility at the CyberGIS Center *Social Media and Viz Team* led by Dr. Su Han and serves the main purpose of preprocessing Covid-19 data by taking the input data of cumulative covid-19 data at county level from New York Times to Metropolitan Statistical Area (MSA) level for further study. This notebook also offers functionality to output global daily covid data and output in js format.

Created at The University of Illinois CyberGIS Center. Created: 12/6/2020. Last updated: 7/31/2021.

## Notebook Outline
- [Data preparation](#Data_preparation)
- [Cumulative to daily](#Cumulative_to_daily)
- [Transpose](#Transpose)
- [Aggregate](#Aggregate)

In [56]:
import sys
import argparse
import os
import pandas as pd
import datetime as dt
from datetime import timedelta
from datetime import datetime
import numpy as np
import json
import geopandas as gpd
from urllib.request import urlopen

<a id = 'Data_preparation'></a>
### Data preparation

In [57]:
# change the parameters below to specify a beginning and an ending date 
startDate = dt.datetime(2020, 2, 14)
endDate = dt.datetime(2021, 12, 8)
inputCovid = './data/Covid-19 cumulative.csv'
inputMetro = './data/metro_county.csv'

# change the parameters below to specify an interval (for example interval = 1
# means daily cases and interval = 7 means weekly cases)
interval = 1

In [58]:
# input cumulative data
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
data = pd.read_csv(url)

# filter out data rows whose dates are not within the range
data["date"] = pd.to_datetime(data["date"])
data = data[data.date <= endDate]
data = data[data.date >= startDate]

data = data.sort_values(by = ['county', 'state'])
copy = data
data.to_csv("./data/daily count.csv", index = False)
data = pd.read_csv("./data/daily count.csv")
copy.to_csv("./data/daily count_copy.csv", index = False)
copy = pd.read_csv("./data/daily count_copy.csv")

In [59]:
# input MSA to county reference table
metro = pd.read_csv(inputMetro)
metro_initial = metro

In [60]:
metro_initial.head()

,states_msa_code,states_msa,states_msa_full,geoid_msa,name_msa,statefp10_county,countyfp10,countyns10,geoid10_county,name10_county,countyid
0,1,AL,AL,33860,Montgomery,1,1,161526,1001,Autauga,1001
1,1,AL,AL,19300,Daphne-Fairhope-Foley,1,3,161527,1003,Baldwin,1003
2,1,AL,AL,13820,Birmingham-Hoover,1,7,161529,1007,Bibb,1007
3,1,AL,AL,13820,Birmingham-Hoover,1,9,161530,1009,Blount,1009
4,1,AL,AL,11500,Anniston-Oxford-Jacksonville,1,15,161533,1015,Calhoun,1015


In [61]:
len(metro_initial)

2340

In [62]:
# input US state abbreviations reference table
state_abbr = pd.read_csv('./data/us states abbreviations.csv')

In [63]:
state_abbr.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [64]:
len(state_abbr)

52

<a id = 'Cumulative_to_daily'></a>
### Cumulative to daily
This section is for converting cumulative cases to daily cases. The basic idea is that the daily case for a specific day is the cumulative cases on the day minus the cumulative cases on the day before it.

In [65]:
i = 1
data['fips'] = data['fips'].fillna(-1)
while i < len(data.index):
    if data.at[i, 'county'] == data.at[i - 1, 'county'] and data.at[i, 'state'] == data.at[i - 1, 'state']:
        if (data.at[i, 'cases'] < copy.at[i - 1, 'cases']):
            data.at[i, 'cases'] = copy.at[i - 1, 'cases']
        if (data.at[i, 'deaths'] < copy.at[i - 1, 'deaths']):
            data.at[i, 'deaths'] = copy.at[i - 1, 'deaths']
        data.at[i, 'cases'] = data.at[i, 'cases'] - copy.at[i - 1, 'cases']
        data.at[i, 'deaths'] = data.at[i, 'deaths'] - copy.at[i - 1, 'deaths']
    i += 1
data = data.astype({'fips': int})
data = data.sort_values(by = ['date'])

data.to_csv("./data/daily count.csv", index = False)

data.head()

,date,county,state,fips,cases,deaths
1077297,2020-02-14,Los Angeles,California,6037,1,0.0
1572467,2020-02-14,San Diego,California,6073,1,0.0
1345737,2020-02-14,Orange,California,6059,1,0.0
1122863,2020-02-14,Maricopa,Arizona,4013,1,0.0
129574,2020-02-14,Bexar,Texas,48029,2,0.0


In [66]:
len(data)

1995196

<a id = 'Aggregate'></a>
### Aggregate
This section is for aggregating

In [67]:
# merge the metro reference table with state reference table
metro_initial = metro_initial.merge(state_abbr, how = 'left', 
                     left_on = 'states_msa', 
                     right_on = 'Abbreviation', indicator = True)
metro_initial['states_msa'] = metro_initial['State']


In [68]:
metro_initial.head()

,states_msa_code,states_msa,states_msa_full,geoid_msa,name_msa,statefp10_county,countyfp10,countyns10,geoid10_county,name10_county,countyid,State,Abbreviation,_merge
0,1,Alabama,AL,33860,Montgomery,1,1,161526,1001,Autauga,1001,Alabama,AL,both
1,1,Alabama,AL,19300,Daphne-Fairhope-Foley,1,3,161527,1003,Baldwin,1003,Alabama,AL,both
2,1,Alabama,AL,13820,Birmingham-Hoover,1,7,161529,1007,Bibb,1007,Alabama,AL,both
3,1,Alabama,AL,13820,Birmingham-Hoover,1,9,161530,1009,Blount,1009,Alabama,AL,both
4,1,Alabama,AL,11500,Anniston-Oxford-Jacksonville,1,15,161533,1015,Calhoun,1015,Alabama,AL,both


In [69]:
len(metro_initial)

2340

In [70]:
metro_initial[metro_initial["_merge"] != "both"]

,states_msa_code,states_msa,states_msa_full,geoid_msa,name_msa,statefp10_county,countyfp10,countyns10,geoid10_county,name10_county,countyid,State,Abbreviation,_merge
2264,72,NaN,PR,10260,Adjuntas,72,1,1804480,72001,Adjuntas,72001,NaN,NaN,left_only
2265,72,NaN,PR,10380,Aguadilla-Isabela,72,3,1804481,72003,Aguada,72003,NaN,NaN,left_only
2266,72,NaN,PR,10380,Aguadilla-Isabela,72,5,1804482,72005,Aguadilla,72005,NaN,NaN,left_only
2267,72,NaN,PR,41980,San Juan-Carolina-Caguas,72,7,1804483,72007,Aguas Buenas,72007,NaN,NaN,left_only
2268,72,NaN,PR,41980,San Juan-Carolina-Caguas,72,9,1804484,72009,Aibonito,72009,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,72,NaN,PR,41980,San Juan-Carolina-Caguas,72,145,1804553,72145,Vega Baja,72145,NaN,NaN,left_only
2336,72,NaN,PR,41980,San Juan-Carolina-Caguas,72,147,1804554,72147,Vieques,72147,NaN,NaN,left_only
2337,72,NaN,PR,38660,Ponce,72,149,1804555,72149,Villalba,72149,NaN,NaN,left_only
2338,72,NaN,PR,41980,San Juan-Carolina-Caguas,72,151,1804556,72151,Yabucoa,72151,NaN,NaN,left_only


In [71]:
metro_initial = metro_initial.drop(columns = ['_merge'])

In [72]:
# merge again with the covid data
merged = metro_initial.merge(data, how = 'right', 
                     left_on = ["states_msa", "name10_county"], 
                     right_on = ["state","county"], indicator = True)
merged = merged[['date', 'county', 'cases', 
                 'deaths', 'name_msa', 'states_msa_code', 'states_msa', 'states_msa_full',
                 'geoid_msa', '_merge']]  


In [73]:
merged.head()

,date,county,cases,deaths,name_msa,states_msa_code,states_msa,states_msa_full,geoid_msa,_merge
0,2020-03-24,Autauga,1,0.0,Montgomery,1.0,Alabama,AL,33860.0,both
1,2020-03-25,Autauga,3,0.0,Montgomery,1.0,Alabama,AL,33860.0,both
2,2020-03-26,Autauga,2,0.0,Montgomery,1.0,Alabama,AL,33860.0,both
3,2020-03-27,Autauga,0,0.0,Montgomery,1.0,Alabama,AL,33860.0,both
4,2020-03-28,Autauga,0,0.0,Montgomery,1.0,Alabama,AL,33860.0,both


In [74]:
merged[merged["_merge"] == "right_only"]

,date,county,cases,deaths,name_msa,states_msa_code,states_msa,states_msa_full,geoid_msa,_merge
1139507,2020-03-01,New York City,1,0.0,NaN,NaN,NaN,NaN,NaN,right_only
1139508,2020-03-02,New York City,0,0.0,NaN,NaN,NaN,NaN,NaN,right_only
1139509,2020-03-03,New York City,1,0.0,NaN,NaN,NaN,NaN,NaN,right_only
1139510,2020-03-04,New York City,0,0.0,NaN,NaN,NaN,NaN,NaN,right_only
1139511,2020-03-05,New York City,2,0.0,NaN,NaN,NaN,NaN,NaN,right_only
...,...,...,...,...,...,...,...,...,...,...
2029569,2021-12-04,Unknown,0,0.0,NaN,NaN,NaN,NaN,NaN,right_only
2029570,2021-12-05,Unknown,0,0.0,NaN,NaN,NaN,NaN,NaN,right_only
2029571,2021-12-06,Unknown,0,0.0,NaN,NaN,NaN,NaN,NaN,right_only
2029572,2021-12-07,Unknown,0,0.0,NaN,NaN,NaN,NaN,NaN,right_only


In [75]:
len(merged)

2029574

In [76]:
# aggregate for each interval of dates
merged["date"] = pd.to_datetime(merged["date"])
iterate_start = startDate

interval_data = merged
output = pd.DataFrame()

while iterate_start <= endDate:
    iterate_end = iterate_start + timedelta(days = interval)

    eachInterval = interval_data[interval_data.date >= iterate_start]
    eachInterval = eachInterval[eachInterval.date < iterate_end]


    eachInterval = eachInterval.groupby(['name_msa', 'states_msa'])['cases', 'deaths'].sum()
    eachInterval = eachInterval.merge(metro_initial, left_on='name_msa', right_on='name_msa')[['states_msa_code', 'states_msa', 
                                'states_msa_full', "geoid_msa",
                               'name_msa', 'cases', 'deaths']].sort_values(by = 'states_msa_code')
    eachInterval['interval_start'] = iterate_start
    eachInterval = eachInterval.drop_duplicates(subset = ['name_msa', 'states_msa'])

    output = output.append(eachInterval)
    iterate_start = iterate_start + timedelta(days=interval)

output.to_csv("./data/output.csv", index = False)
output.head()


<ipython-input-76-5840767d63bf>:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  eachInterval = eachInterval.groupby(['name_msa', 'states_msa'])['cases', 'deaths'].sum()


,states_msa_code,states_msa,states_msa_full,geoid_msa,name_msa,cases,deaths,interval_start
63,4,Arizona,AZ,38060,Phoenix-Mesa-Scottsdale,1,0.0,2020-02-14
80,6,California,CA,41940,San Jose-Sunnyvale-Santa Clara,2,0.0,2020-02-14
78,6,California,CA,41860,San Francisco-Oakland-Hayward,2,0.0,2020-02-14
73,6,California,CA,41740,San Diego-Carlsbad,1,0.0,2020-02-14
57,6,California,CA,31080,Los Angeles-Long Beach-Anaheim,2,0.0,2020-02-14


<a id = 'Transpose'></a>
### Transpose
Here we transpose the aggregated output from the previous cell so that each date is its own column and we will have a separate output cases and output deaths data

#### transpose MSA

In [77]:
# initilize some dictionaries to be used for the transposition
geoid_msa = {}
MSA_all_cases = {}
MSA_all_deaths = {}
all_dates = {}

for index, row in output.iterrows():
    MSA_all_cases[row['name_msa'] + '/' + row['states_msa_full']] = []
    MSA_all_deaths[row['name_msa'] + '/' + row['states_msa_full']] = []
    all_dates[row['interval_start']] = 0
    geoid_msa[row['geoid_msa']] = 0

for index, row in output.iterrows():
    MSA_all_cases[row['name_msa'] + '/' + row['states_msa_full']].append(row['cases'])
    MSA_all_deaths[row['name_msa'] + '/' + row['states_msa_full']].append(row['deaths'])

In [78]:
print(len(MSA_all_cases), len(geoid_msa))

970 899


In [79]:
print(MSA_all_cases['AberdeenSD'])

KeyError: 'AberdeenSD'

In [80]:
# filling up the dictionaries so that the keys are MSA names and the values
# will be the cases/deaths on each date
MSA_all_cases_list = []
for value in MSA_all_cases.values():
    MSA_all_cases_list.append(value)

MSA_all_deaths_list = []
for value in MSA_all_deaths.values():
    for index, v in enumerate(value):
        value[index] = int(v)
    MSA_all_deaths_list.append(value)

for i in MSA_all_cases_list:
    if (len(i) < len(all_dates)):
        diff = len(all_dates) - len(i)
        for j in range(diff):
            i.insert(j, 0)

for i in MSA_all_deaths_list:
    if (len(i) < len(all_dates)):
        diff = len(all_dates) - len(i)
        for j in range(diff):
            i.insert(j, 0)

In [81]:
# format the dates into strings
dates = list(all_dates.keys())
for i in range(len(dates)):
    dates[i] = str(dates[i])[:-9]

In [82]:
# convert the dictionaries back to lists for dataframe constructions
output_cases = pd.DataFrame(MSA_all_cases_list, columns = dates)
print(output_cases.shape)
print(len(list(geoid_msa.keys())))
#output_cases.insert(0, 'geoid', list(geoid_msa.keys()))
output_cases.insert(0, 'name', list(MSA_all_cases.keys()))

output_deaths = pd.DataFrame(MSA_all_deaths_list, columns = dates)
#output_deaths.insert(0, 'geoid', list(geoid_msa.keys()))
output_deaths.insert(0, 'name', list(MSA_all_deaths.keys()))

(970, 664)
899


In [83]:
# add the state column and remove the state name from the MSA name column
states = []
i = 0
while i < len(output_cases.index):
    state = output_cases.at[i, 'name'].split('/')[1]
    states.append(state)
    i+=1
    
    
i = 0
while i < len(output_cases.index):
    output_cases.at[i, 'name'] = output_cases.at[i, 'name'].split('/')[0]
    i+=1

i = 0
while i < len(output_deaths.index):
    output_deaths.at[i, 'name'] = output_deaths.at[i, 'name'].split('/')[0]
    i+=1

# print(len(states))
output_cases.insert(1, 'state', states)
output_deaths.insert(1, 'state', states)

In [84]:
output_deaths.head()

,name,state,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,...,2021-11-29,2021-11-30,2021-12-01,2021-12-02,2021-12-03,2021-12-04,2021-12-05,2021-12-06,2021-12-07,2021-12-08
0,Phoenix-Mesa-Scottsdale,AZ,0,0,0,0,0,0,0,0,...,0,48,21,16,5,88,27,0,93,0
1,San Jose-Sunnyvale-Santa Clara,CA,0,0,0,0,0,0,0,0,...,0,5,5,0,3,0,0,5,4,4
2,San Francisco-Oakland-Hayward,CA,0,0,0,0,0,0,0,0,...,2,0,6,6,13,0,0,3,1,2
3,San Diego-Carlsbad,CA,0,0,0,0,0,0,0,0,...,6,10,5,6,4,0,0,1,1,5
4,Los Angeles-Long Beach-Anaheim,CA,0,0,0,0,0,0,0,0,...,45,43,29,37,24,19,2,17,26,26


In [85]:
output_cases.head()

,name,state,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,...,2021-11-29,2021-11-30,2021-12-01,2021-12-02,2021-12-03,2021-12-04,2021-12-05,2021-12-06,2021-12-07,2021-12-08
0,Phoenix-Mesa-Scottsdale,AZ,1,0,0,0,0,0,0,0,...,1305,2018,2231,2644,3502,4012,2646,2033,2218,2530
1,San Jose-Sunnyvale-Santa Clara,CA,2,0,0,0,0,0,0,0,...,784,192,193,306,333,0,0,899,203,176
2,San Francisco-Oakland-Hayward,CA,2,0,0,0,0,0,0,0,...,1422,256,243,704,570,0,0,1757,353,345
3,San Diego-Carlsbad,CA,1,0,0,0,0,0,0,0,...,391,1828,552,1049,654,0,0,1269,2410,721
4,Los Angeles-Long Beach-Anaheim,CA,2,0,0,0,0,0,0,0,...,2012,1082,1682,2254,2260,2290,1631,2542,1307,2032


In [86]:
output_cases.to_csv("./data/output_cases.csv", index = False)
output_deaths.to_csv("./data/output_deaths.csv", index = False)

#### transpose world

In [36]:
# input global covid data
input_world = './data/covid_world.csv'
df = pd.read_csv(input_world)
df["date"] = pd.to_datetime(df["date"])
df = df[df.date <= endDate]
df = df[df.date >= startDate]
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


In [37]:
# geoid data for reference
geoid = pd.read_csv('./data/geoid.csv')
geoid.head()

,Geographical location identifier (Hex),Geographical location identifier (decimal),Location (Short Name),Location (Long Name)
0,0x2,2,Antigua and Barbuda,Antigua and Barbuda
1,0x3,3,Afghanistan,Islamic Republic of Afghanistan
2,0x4,4,Algeria,Democratic and Popular Republic of Algeria
3,0x5,5,Azerbaijan,Republic of Azerbaijan
4,0x6,6,Albania,Republic of Albania


In [38]:
# merge the geoid and world data 
df = df.merge(geoid, how = 'inner', 
                     left_on = ['location'], 
                     right_on = ['Location (Short Name)'] )

df = df[['Geographical location identifier (decimal)', 'iso_code', 'date', 'continent', 'location', 
         'new_cases', 'new_deaths']]

In [39]:
# initilize some dictionaries to be used for the transposition
iso_country = {}
country_all_cases = {}
country_all_deaths = {}
all_dates = {}
continent = {}
locations = {}
geoid = {}

# for i in range(df.shape[0]):
#     transform = df.at[i, 'date']
#     print(transform)
#     transform = transform.split('/')
#     transform = transform[0] + '-' + transform[1] + '-' + transform[2]


for i in range(df.shape[0]):
    if (df.at[i, 'date'] < startDate or df.at[i, 'date'] > endDate):
        df = df.drop([i])

for index, row in df.iterrows():
    country_all_cases[row['iso_code']] = []
    country_all_deaths[row['iso_code']] = []
    all_dates[row['date']] = 0
    iso_country[row['iso_code']] = 0

    continent[row['continent']] = 0
    locations[row['location']] = 0
    geoid[row['Geographical location identifier (decimal)']] = 0

    

In [40]:
# filling up the dictionaries so that the keys are iso_code for each country
# and the values are cases/deaths on each date
for index, row in df.iterrows():
    country_all_cases[row['iso_code']].append(row['new_cases'])
    country_all_deaths[row['iso_code']].append(row['new_deaths'])

country_all_cases_list = []
for value in country_all_cases.values():
    country_all_cases_list.append(value)

country_all_deaths_list = []
for value in country_all_deaths.values():
    country_all_deaths_list.append(value)


dates = list(all_dates.keys())
for i in range(len(dates)):
    dates[i] = str(dates[i])[:-9]


output_cases_world = pd.DataFrame(country_all_cases_list, columns = dates)
output_cases_world.insert(0, 'geoid', list(geoid.keys()))
output_cases_world.insert(1, 'name', list(locations.keys()))


output_deaths_world = pd.DataFrame(country_all_deaths_list, columns = dates)
output_deaths_world.insert(0, 'geoid', list(geoid.keys()))
output_deaths_world.insert(1, 'name', list(locations.keys()))

output_cases_world.head()

,geoid,name,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,...,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23
0,3,Afghanistan,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Algeria,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,Andorra,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,Angola,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
output_cases.head()

,geoid,name,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,...,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,2021-09-14,2021-09-15,2021-09-16
0,38060,Phoenix-Mesa-Scottsdale,1,0,0,0,0,0,0,0,...,1446,1766,1739,2150,2388,2091,1681,1941,1802,1877
1,41940,San Jose-Sunnyvale-Santa Clara,2,0,0,0,0,0,0,0,...,325,182,232,201,324,342,248,268,189,187
2,41860,San Francisco-Oakland-Hayward,2,0,0,0,0,0,0,0,...,442,505,1066,712,973,844,755,577,522,728
3,41740,San Diego-Carlsbad,1,0,0,0,0,0,0,0,...,795,487,728,712,1024,1182,703,589,557,729
4,31080,Los Angeles-Long Beach-Anaheim,2,0,0,0,0,0,0,0,...,2832,2405,2534,2630,0,0,7362,2137,2298,2312


In [ ]:
output_cases_world.columns[len(output_cases_world.columns) - 1]

In [41]:
# merge the world output and MSA output
for i in range(output_cases_world.shape[0]):
    output_cases.loc[output_cases.shape[0] + i] = list(output_cases_world.loc[i])

for i in range(output_deaths_world.shape[0]):
    output_deaths.loc[output_deaths.shape[0] + i] = list(output_deaths_world.loc[i])

output_cases.to_csv("./data/output_cases.csv", index = False)
output_deaths.to_csv("./data/output_deaths.csv", index = False)

output_cases.head()

ValueError: cannot set a row with mismatched columns